In [0]:
# Which disease has a maximum number of claims.
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.access.key", "AK")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "SK")

from pyspark.sql.functions import col, current_date, datediff, floor



In [0]:
subscribers = spark.read.options(header=True, inferSchema=True, delimiter = ',').csv("s3://capstone-kanchan/healthcare/subscriber.csv")
subscribers = subscribers.dropDuplicates()
subscribers = subscribers.na.fill("NA")
subscribers = subscribers.withColumn("Age", floor(datediff(current_date(),col("Birth_date"))/365.25))
subscribers = subscribers.filter((subscribers.Age <30) & (subscribers.Subgrp_id != "NA"))
subscribers = subscribers.select("sub _id","first_name","last_name","Subgrp_id")
subscribers = subscribers.withColumnRenamed("sub _id", "sub_id")


In [0]:
subscribers.write.format("redshift").option("url", "jdbc:redshift://5439/dev").\
   option("dbtable", "capstone.subscribersLess30").\
   option("aws_iam_role", "arn:aw:role/Redshift_admin").\
   option("tempdir", "s3a://capstone-kanchan/temp/").\
   option("user", "admin").\
   option("password", "PW").save()